In [1]:
# !pip install --upgrade tensorflow
!pip install tf-quant-finance
# !pip install tqdm

You should consider upgrading via the 'C:\Dev\github\tf-quant-finance\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
#@title Imports { display-mode: "form" }

import matplotlib.pyplot as plt
import numpy as np
import time

import tensorflow as tf

 # tff for Tensorflow Finance
import tf_quant_finance as tff

from IPython.core.pylabtools import figsize
figsize(21, 14) # better graph size for Colab  

from tqdm.notebook import tqdm, trange
from QuantLib import *

In [3]:
#@title Set up parameters

dtype = np.float64 #@param
num_samples = 50000 #@param
num_timesteps = 50 #@param

def pricer_eu_options(expiries, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price(strikes, spot, sigma):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            expiries, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.PSEUDO_ANTITHETIC,
            time_step=dt)
        prices = (tf.exp(-tf.expand_dims(rate * expiries, axis=-1))
                  * tf.reduce_mean(tf.nn.relu(strikes-tf.math.exp(paths)), 0))
        return prices
    return price

In [4]:
expiries = [1.0] # This can be a rank 1 Tensor
dt = 1. / num_timesteps
rate = tf.constant(0.06, dtype=dtype)
sigma = tf.constant(0.2, dtype=dtype)
spot = tf.constant(36.0, dtype=dtype)
strikes = tf.constant(40.0, dtype=dtype)

price_eu_options = tf.function(pricer_eu_options(expiries),
                    input_signature=[tf.TensorSpec([], dtype=tf.float64),
                     tf.TensorSpec([], dtype=tf.float64),
                     tf.TensorSpec([], dtype=tf.float64)])

price_eu_options(strikes, spot, sigma).numpy()[0][0]

3.8340222821454817

In [5]:
tqf_BS_price = tff.black_scholes.option_price(
      volatilities=sigma,
      strikes=strikes,
      expiries=expiries,
      spots=spot,
      discount_rates=rate,
      dividend_rates=tf.constant(0.0,tf.float64),
      is_call_options=False
      ).numpy()[0]
print(tqf_BS_price)

3.8443077915968416


In [9]:
def pricer_am_options(times, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price(strikes, spot, sigma):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma]
        else:
            watch_params_list = None
            
        paths = process.sample_paths(
            times=times, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            random_type=tff.math.random.RandomType.SOBOL, #PSEUDO_ANTITHETIC
            time_step=dt)
        
        exercise_times = tf.range(times.shape[-1])
        payoff_fn = tff.models.longstaff_schwartz.make_basket_put_payoff(strikes, dtype=tf.float64)
        basis_fn = tff.models.longstaff_schwartz.make_polynomial_basis(10)
        
        prices = tff.models.longstaff_schwartz.least_square_mc(
            tf.math.exp(paths), exercise_times, payoff_fn, basis_fn,
            discount_factors=tf.exp(-rate * times),
            dtype=dtype)
        
        return prices
    return price

times = tf.linspace(tf.constant(0.0, dtype=dtype), tf.constant(expiries[0], dtype=dtype), num_timesteps)
price_am_options = tf.function(pricer_am_options(times),
                    input_signature=[tf.TensorSpec([], dtype=tf.float64),
                     tf.TensorSpec([], dtype=tf.float64),
                     tf.TensorSpec([], dtype=tf.float64)])

t = time.time()
tqf_price = price_am_options(strikes, spot, sigma).numpy()[0]
time_tqf = time.time() - t

print('------------------------')
print('TQF American Put')
print('wall time: ', time_tqf)
print('options per second: ', 1.0/time_tqf)
print('------------------------')
print('TQF American put price', tqf_price)


t = time.time()
tqf_price = price_am_options(strikes, spot, sigma).numpy()[0]
time_tqf = time.time() - t

print('------------------------')
print('TQF American Put')
print('wall time: ', time_tqf)
print('options per second: ', 1.0/time_tqf)
print('------------------------')
print('TQF American put price', tqf_price)

------------------------
TQF American Put
wall time:  2.86789870262146
options per second:  0.3486873504583443
------------------------
TQF American put price 4.0
------------------------
TQF American Put
wall time:  0.46756887435913086
options per second:  2.1387223462439438
------------------------
TQF American put price 4.0


In [7]:
tqf_amer_price = tff.black_scholes.approximations.bjerksund_stensland(
      volatilities=sigma,
      strikes=strikes,
      expiries=expiries,
      spots=spot,
      discount_rates=rate,
      dividend_rates=tf.constant(0.0,tf.float64),
      is_call_options=False
      ).numpy()[0]
print(tqf_amer_price)

4.471186097072536


In [8]:
def ql_american_price(calc_date, settlement_date, expiry_date, spot, strike, rf, div, vol):
    Settings.instance().evaluationDate = calc_date
    riskFreeRate = FlatForward(settlement_date, rf, Actual365Fixed())
    exercise = AmericanExercise(calc_date, expiry_date)
    payoff = PlainVanillaPayoff(Option.Put, strike)
    underlying = SimpleQuote(spot)
    volatility = BlackConstantVol(calc_date, TARGET(), vol, Actual365Fixed())
    dividendYield = FlatForward(settlement_date, div, Actual365Fixed())
    process = BlackScholesMertonProcess(QuoteHandle(underlying),
                                        YieldTermStructureHandle(dividendYield),
                                        YieldTermStructureHandle(riskFreeRate),
                                        BlackVolTermStructureHandle(volatility))
    option = VanillaOption(payoff, exercise)
    option.setPricingEngine(MCAmericanEngine(process, 'PseudoRandom', 
                             timeSteps=num_timesteps, 
                             polynomOrder=10, 
                             seedCalibration=42, 
                             requiredSamples=num_samples))
    closedFormulaNPV = option.NPV()
    return closedFormulaNPV

t = time.time()
ql_price = ql_american_price(
        calc_date=Date(15,May,1998),
        settlement_date=Date(15,May,1998),
        expiry_date=Date(15,May,1999),
        spot=spot.numpy(),
        strike=40.0,
        rf=rate.numpy(),
        div=0.0,
        vol=sigma.numpy()
    )
time_ql = time.time() - t

print('------------------------')
print('QL American Put')
print('wall time: ', time_ql)
print('options per second: ', 1.0/time_ql)
print('------------------------')
print('Quantlib American put price', ql_price)

------------------------
QL American Put
wall time:  0.8819422721862793
options per second:  1.1338610604536088
------------------------
Quantlib American put price 4.451439442214406
